Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI pipeline with PyTorch 2.0 Features

This notebook introduces how to use `torch.compile` in the MONAI pipeline. `torch.compile` is the main API for PyTorch 2.0, which wraps your model and returns a compiled model. It is a fully additive (and optional) feature and hence 2.0 is 100% backward compatible by definition. We also run an end-to-end pipeline based on ["fast_training_tutorial.ipynb"](https://github.com/Project-MONAI/tutorials/blob/main/acceleration/fast_training_tutorial.ipynb), and the speed up is 1.16x.

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, matplotlib]"
!pip install -q torch>=2.1.0

## Setup imports

In [ ]:
import os
import time
import torch
import tempfile
import numpy as np

import monai
import monai.transforms as mt
from monai.config import print_config
from monai.utils import set_determinism
from monai.bundle import download, create_workflow
from monai.engines import SupervisedTrainer

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified, a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/data


## Download dataset

In [4]:
sample_url = "https://github.com/Project-MONAI/MONAI-extra-test-data/releases"
sample_url += "/download/0.8.1/totalSegmentator_mergedLabel_samples.zip"
monai.apps.download_and_extract(sample_url, output_dir=root_dir, filepath="samples.zip")

base_name = os.path.join(root_dir, "totalSegmentator_mergedLabel_samples")
input_data = []
for filename in os.listdir(os.path.join(base_name, "imagesTr")):
    input_data.append(
        {
            "image": os.path.join(base_name, "imagesTr", filename),
            "label": os.path.join(base_name, "labelsTr", filename),
        }
    )

2024-01-16 05:55:56,446 - INFO - Expected md5 is None, skip md5 check for file samples.zip.
2024-01-16 05:55:56,447 - INFO - File exists: samples.zip, skipped downloading.
2024-01-16 05:55:56,448 - INFO - Writing into directory: /workspace/data.


## Set deterministic for reproducibility

In [5]:
set_determinism(seed=0)

## Set up timing and training functions

For best accuracies, we use CUDA events and synchronization to measure the forward and backward propagations in training.

In [6]:
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000


def train(model, inputs, labels):
    outputs = model(inputs)
    loss_function = monai.losses.DiceCELoss(to_onehot_y=True, softmax=True)
    loss = loss_function(outputs, labels)
    loss.backward()
    return loss

## Set up model

Here we used `create_workflow` to get the network instance from the bundle. You can also initialize your own network.

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
bundle_dir = "./bundle"
os.makedirs(bundle_dir, exist_ok=True)

bundle = download("wholeBody_ct_segmentation", bundle_dir=bundle_dir)
config_file = os.path.join(bundle_dir, "wholeBody_ct_segmentation/configs/train.json")
train_workflow = create_workflow(config_file=str(config_file), workflow_type="train")


def init_model(device):
    return train_workflow.network_def.to(device)

2024-01-16 05:55:59,615 - INFO - --- input summary of monai.bundle.scripts.download ---
2024-01-16 05:55:59,617 - INFO - > name: 'wholeBody_ct_segmentation'
2024-01-16 05:55:59,617 - INFO - > bundle_dir: './bundle'
2024-01-16 05:55:59,618 - INFO - > source: 'monaihosting'
2024-01-16 05:55:59,619 - INFO - > remove_prefix: 'monai_'
2024-01-16 05:55:59,619 - INFO - > progress: True
2024-01-16 05:55:59,620 - INFO - ---


2024-01-16 05:55:59,797 - INFO - Expected md5 is None, skip md5 check for file bundle/wholeBody_ct_segmentation_v0.2.1.zip.
2024-01-16 05:55:59,798 - INFO - File exists: bundle/wholeBody_ct_segmentation_v0.2.1.zip, skipped downloading.
2024-01-16 05:55:59,799 - INFO - Writing into directory: bundle.
2024-01-16 05:56:00,729 - INFO - --- input summary of monai.bundle.scripts.run ---
2024-01-16 05:56:00,729 - INFO - > config_file: './bundle/wholeBody_ct_segmentation/configs/train.json'
2024-01-16 05:56:00,730 - INFO - > workflow_type: 'train'
2024-01-16 05:56:00,731 - INFO - 

## Set up DataLoader and train transforms

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transform = mt.Compose(
    [
        mt.LoadImageD(keys=("image", "label"), image_only=True, ensure_channel_first=True),
        mt.SpacingD(keys=("image", "label"), pixdim=1.5),
        mt.EnsureTypeD(keys=("image", "label"), device=device),
        mt.RandRotateD(
            keys=("image", "label"),
            prob=1.0,
            range_x=0.1,
            range_y=0.1,
            range_z=0.3,
            mode=("bilinear", "nearest"),
        ),
        mt.RandZoomD(keys=("image", "label"), prob=1.0, min_zoom=0.8, max_zoom=1.2, mode=("trilinear", "nearest")),
        mt.ResizeWithPadOrCropD(keys=("image", "label"), spatial_size=(96, 96, 96)),
        # add `FromMetaTensorD` to convert `MetaTensor` to `torch.Tensor`
        mt.FromMetaTensorD(keys=("image", "label")),
    ]
)

dataset = monai.data.CacheDataset(data=input_data, transform=transform, cache_rate=1.0, num_workers=4)
data_loader = monai.data.DataLoader(dataset, batch_size=1)

Loading dataset: 100%|██████████| 20/20 [00:05<00:00,  3.85it/s]


## Training in eager mode

In [9]:
# start a typical PyTorch training
epoch_num = 20
model = init_model(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

epoch_loss_values = []
eager_time = []
for epoch in range(epoch_num):
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in data_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        loss, train_time = timed(lambda: train(model, inputs, labels))
        optimizer.step()
        epoch_loss += loss.item()
        eager_time.append(train_time)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    if epoch < 5:
        print(f"epoch {epoch + 1}/{epoch_num} average loss: {epoch_loss:.4f}, time: {train_time:.4f}")
print(f"median training time per iteration: {np.median(eager_time):.4f} seconds")

epoch 1/20 average loss: 5.3744, time: 0.0841
epoch 2/20 average loss: 4.9033, time: 0.0835
epoch 3/20 average loss: 4.2363, time: 0.0837
epoch 4/20 average loss: 3.7749, time: 0.0838
epoch 5/20 average loss: 3.4824, time: 0.0844
median training time per iteration: 0.0838 seconds


## Train the model with torch.compile

The only difference is that we wrap the model with `torch.compile`. You may also see different speedup results depending on the chosen mode argument. The "reduce-overhead" mode uses CUDA graphs to further reduce the overhead of Python. For your own models, you may need to experiment with different modes to maximize speedup. You can read more about modes [here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).

In [10]:
model_opt = torch.compile(init_model(device), mode="reduce-overhead")

In [11]:
# start a typical PyTorch training
optimizer = torch.optim.Adam(model_opt.parameters(), 1e-3)

epoch_loss_values = []
compile_time = []
for epoch in range(epoch_num):
    model_opt.train()
    epoch_loss = 0
    step = 0
    for batch_data in data_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        loss, train_time = timed(lambda: train(model_opt, inputs, labels))
        optimizer.step()
        epoch_loss += loss.item()
        compile_time.append(train_time)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    if epoch < 5:
        print(f"epoch {epoch + 1}/{epoch_num} average loss: {epoch_loss:.4f}, time: {train_time:.4f}")
print(f"median training time per iteration after compilation: {np.median(compile_time):.4f} seconds")

Using FallbackKernel: aten.upsample_trilinear3d
Using FallbackKernel: aten.upsample_trilinear3d_backward


epoch 1/20 average loss: 3.0229, time: 0.0529
epoch 2/20 average loss: 2.9459, time: 0.0530
epoch 3/20 average loss: 2.9603, time: 0.0529
epoch 4/20 average loss: 2.9253, time: 0.0529
epoch 5/20 average loss: 2.8783, time: 0.0531
median training time per iteration after compilation: 0.0529 seconds


## Demonstrate the speedups
As [torch](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html) mentioned, we can see that `torch.compile` takes longer in the first epoch, as it must compile the model, but in subsequent iterations, we can see a significant improvement compared to eager. 

In [16]:
print(f"first iteration: {eager_time[0]:.3f}s(eager mode) vs. {compile_time[0]:.3f}s(compile mode)")

speedup = np.median(eager_time) / np.median(compile_time)
print(f"eager median: {np.median(eager_time)}, compile median: {np.median(compile_time)}, speedup: {speedup}x")

first iteration: 2.767s(eager mode) vs. 22.391s(compile mode)
eager median: 0.08376780700683595, compile median: 0.05292851257324219, speedup: 1.5826593821414972x


We also tried `torch.compile` in [fast_training_tutorial.ipynb](https://github.com/Project-MONAI/tutorials/blob/main/acceleration/fast_training_tutorial.ipynb).
The total training time for fast and compile is as follows: 354.9534s and 305.6460s, speedup: 1.16x.

![compile_benchmark_total_epoch_time_comparison](../figures/total_epoch_time_comparison-compile.png)

## Use the `torch.compile` in the bundle

We can simply set `compile=True` in the `SupervisedTrainer` and `SupervisedEvaluator`. Here we convert data to `torch.Tensor` internally if set `compile=True`. Here is the [ticket](https://github.com/pytorch/pytorch/issues/117026) we can track.

In [ ]:
loss_function = monai.losses.DiceCELoss(to_onehot_y=True, softmax=True)
trainer = SupervisedTrainer(
    device=device,
    max_epochs=epoch_num,
    train_data_loader=data_loader,
    network=model,
    optimizer=optimizer,
    loss_function=loss_function,
    # postprocessing=post_transform,
    # amp=args.amp,
    # key_train_metric={
    #     "train_dice": MeanDice(
    #         include_background=False,
    #         output_transform=from_engine(["pred", "label"]),
    #     )
    # },
    compile=True,
    # you can also add `compile_kwargs` dict of the args for `torch.compile()` API
    compile_kwargs={},
)